# Check whether new initial and boundary conditions are reasonable

In [4]:
import xarray as xr
import numpy as np
import cmocean
import sys
sys.path.append('/home/users/birgal/')
import pickle
from nemo_python_git.utils import fix_lon_range
from nemo_python_git.interpolation import regrid_array_cf, regrid_operator_cf
from nemo_python_git.ics_obcs import fill_ocean

In [5]:
base_folder = '/gws/nopw/j04/anthrofail/birgal/NEMO_AIS/'

In [ ]:
# Settings:
base_folder = '/gws/nopw/j04/anthrofail/birgal/NEMO_AIS/'
year = 1850
variable = 'ConsTemp'
cesm2var = 'TEMP'
varname  = 'ConsTemp'
gtype    = 'T'

# load NEMO eANT025 grids
domcfg   = xr.open_dataset(f'{base_folder}bathymetry/domain_cfg-20240305.nc')
meshmask = xr.open_dataset(f'{base_folder}bathymetry/mesh_mask-20240305.nc')

# create xarray dataset of cesm2 forcing
climate_forcing = f'{base_folder}climate-forcing/CESM2/LE2/processed/'
cesm2_ds        = xr.open_dataset(f'{climate_forcing}CESM2-LE2_ens1011.001_{cesm2var}_y{year}.nc')
cesm2_ds['z_t'] = cesm2_ds['z_t']*1e-2
cesm2_ds        = cesm2_ds.rename({'TLONG':'lon', 'TLAT':'lat'})
cesm2_lon_E     = fix_lon_range(cesm2_ds['lon'])
cesm2_ds['lon'] = cesm2_lon_E


In [ ]:
# adjust loop depending on the year because the ocean data starts in Feb. 1850
if year==1850:
    month_range = [2,12]#[2,12]
else:
    month_range = [1,12]

for month in range(month_range[0],month_range[1]+1):
    cesm2_source = cesm2_ds.isel(time=(cesm2_ds.time.dt.month==month)).isel(time=0).assign({'depth':cesm2_ds.z_t}).rename({varname:'data', 'nlat':'y', 'nlon':'x', 'z_t':'z'})

    # load interpolation pickles
    with open(f'{base_folder}cf-regridding/linear-regrid-CESM2-eANT025-z0-z70.pickle','rb') as file:
        rgo1 = pickle.load(file)
        ds1  = regrid_array_cf(cesm2_source, rgo1, key_3d=True, method='linear', src_cyclic=True, dst_cyclic=True)

    with open(f'{base_folder}cf-regridding/linear-regrid-CESM2-eANT025-z70-z121.pickle','rb') as file:
        rgo2 = pickle.load(file)
        ds2  = regrid_array_cf(cesm2_source, rgo2, key_3d=True, method='linear', src_cyclic=True, dst_cyclic=True)

    # combine pickles from both interpolations
    ds_array        = np.vstack((ds1.array, ds2.array))
    ds_array_masked = xr.where(ds_array==0, np.nan, ds_array)

    # extract outer edge for boundary conditions
    ds_bdy = ds_array_masked[:,slice(-1,None),:]

    # Fill areas that are masked in source dataset but not in NEMO with nearest neighbours:
    CESM2_interp   = xr.Dataset({variable:(('z','y','x'), ds_bdy)})
    CESM2_extended = fill_ocean(CESM2_interp, variable, meshmask.isel(y=slice(-2,-1)), dim='3D', niter=400, fill_val=np.nan, grid=gtype)
    CESM2_extended[variable] = CESM2_extended[variable].roll(x=-1) # because of fortran based indexing

    # Fill nan values with an unrealistic constant value and correct shape to match the expected shape
    CESM2_extended[variable] = xr.where(np.isnan(CESM2_extended[variable]), 9999, CESM2_extended[variable])
    CESM2_extended[variable] = xr.where(CESM2_extended.z == CESM2_extended.z[-1], CESM2_extended[variable].isel(z=-2), CESM2_extended[variable])
    CESM2_extended[variable] = ('time_counter','deptht','y','x'), CESM2_extended[variable].values[np.newaxis, ...]

    # Write output to file:
    CESM2_extended.to_netcdf(f'{base_folder}boundary-conditions/CESM2/LE2/CESM2_boundary_{variable}_y{year}m{month:02}.nc', unlimited_dims='time_counter')

In [ ]:
# var='ConsTemp'
base_folder = '/gws/nopw/j04/anthrofail/birgal/NEMO_AIS/'
# var = 'AbsSal'
# old_ics = xr.open_dataset(f'{base_folder}initial-conditions/SOSE-{var}-initial-conditions-20240507.nc').isel(time_counter=0)
# new_ics = xr.open_dataset(f'{base_folder}initial-conditions/CESM2_initial_{var}_y1850.nc').isel(time_counter=0)
# old_bcs = xr.open_dataset(f'{base_folder}boundary-conditions/B-SOSE/SOSE-{var}-boundary-conditions-20240507_m01.nc')
# new_bcs = xr.open_dataset(f'{base_folder}boundary-conditions/CESM2/LE2/CESM2_boundary_{var}_y1850m02.nc').isel(time_counter=0)

old_bcs = xr.open_dataset(f'{base_folder}boundary-conditions/B-SOSE/SOSE-SeaIceArea-boundary-conditions-20240507_m09.nc')
new_bcs = xr.open_dataset(f'{base_folder}boundary-conditions/CESM2/LE2/CESM2_boundary_aice_y1850m09.nc').isel(time_counter=0)

In [ ]:
new_bcs = xr.open_mfdataset(f'{base_folder}boundary-conditions/CESM2/LE2/CESM2_boundary_aice_y1850m*', concat_dim='t', combine='nested')

In [ ]:
new_bcs.aice.isel(time_counter=0,y=0).max().values

In [ ]:
old_bcs['x'] = new_bcs.x
old_bcs.SIarea.plot(figsize=(14,5))

In [ ]:
new_bcs.aice.plot(figsize=(14,5))

In [ ]:
differences1 = xr.where((old_bcs['VVEL']==9999) * (new_bcs['VVEL']!=9999), 10, 0)
differences1['x'] = new_bcs.x
differences1.plot(figsize=(18,5))

In [ ]:
differences1 = xr.where((old_ics[var].isel(y=-1)==9999) * (old_bcs[var]!=0), 10, 0)
differences1['x'] = new_bcs.x
differences1.plot(figsize=(18,5))

In [ ]:
differences2 = xr.where((new_ics[var].isel(y=-1)==9999) * (new_bcs[var]!=9999), 10, 0)
differences2.plot(figsize=(18,5), vmin=0, vmax=1)

In [ ]:
(differences2-differences1).plot(figsize=(18,5), vmin=-0.1, vmax=0.1, cmap=cmocean.cm.balance)

In [ ]:
(new_ics.AbsSal-old_ics.AbsSal).max()

In [ ]:
(new_ics.ConsTemp-old_ics.ConsTemp).max()

In [ ]:
new_ics.AbsSal.isel(deptht=0).plot(ylim=(440, 454), figsize=(18,4), vmin=32, vmax=35)

## Organize functions for new CESM2 initial conditions creation

In [6]:
import xarray as xr
import numpy as np
import sys
sys.path.append('/home/users/birgal/')
import pickle
from nemo_python_git.utils import fix_lon_range, convert_to_teos10
from nemo_python_git.interpolation import regrid_array_cf, regrid_operator_cf
from nemo_python_git.ics_obcs import fill_ocean
from nemo_python_git.forcing import find_cesm2_file

In [7]:
# Helper function
def fill_and_reshape(ds, variable, land_fill_val=9999, dim='3D'):
    # Fill nan values with an unrealistic constant value
    ds[variable] = xr.where(np.isnan(ds[variable]), land_fill_val, ds[variable])

    # Reshape array
    if dim=='3D':
        ds[variable] = xr.where(ds.z == ds.z[-1], ds[variable].isel(z=-2), ds[variable]) # fill bottom grid cell with value from above it
        ds[variable] = ('time_counter','deptht','y','x'), ds[variable].values[np.newaxis, ...] # add a time axis
    elif dim=='2D':
        ds[variable] = ('time_counter','y','x'), ds[variable].values[np.newaxis, ...] # add a time axis           
    return ds

In [8]:
def regrid_cesm2_nemo_cf(cesm2_source, variable, dim='3D'):

    if dim=='3D':
        with open('/gws/nopw/j04/anthrofail/birgal/NEMO_AIS/cf-regridding/linear-regrid-CESM2-eANT025-z0-z70.pickle','rb') as file:
            rgo1 = pickle.load(file)
            ds1  = regrid_array_cf(cesm2_source, rgo1, key_3d=True, method='linear', src_cyclic=True, dst_cyclic=True)
        with open('/gws/nopw/j04/anthrofail/birgal/NEMO_AIS/cf-regridding/linear-regrid-CESM2-eANT025-z70-z121.pickle','rb') as file:
            rgo2 = pickle.load(file)
            ds2  = regrid_array_cf(cesm2_source, rgo2, key_3d=True, method='linear', src_cyclic=True, dst_cyclic=True)
        # combine pickles from both interpolations
        ds_array = np.vstack((ds1.array, ds2.array))
    elif dim=='2D':
        with open('/gws/nopw/j04/anthrofail/birgal/NEMO_AIS/cf-regridding/linear-regrid-CESM2-eANT025-z0.pickle','rb') as file:
            rgo1 = pickle.load(file)
            ds1  = regrid_array_cf(cesm2_source, rgo1, key_3d=False, method='linear', src_cyclic=True, dst_cyclic=True)
            ds_array = ds1.array

    # Replace zeros with NaN
    ds_array_masked = xr.where(ds_array==0, np.nan, ds_array)
    if dim=='3D':
        ds = xr.Dataset({variable:(('z','y','x'), ds_array_masked)})
    elif dim=='2D':
        ds = xr.Dataset({variable:(('y','x'), ds_array_masked)})

    return ds

In [32]:
def cesm2_bcs(variable, exp='LE2', ens='1011.001', dim='3D', year=1850, land_fill_val=9999, gtype='T',
              cesm2_folder='/gws/nopw/j04/anthrofail/birgal/NEMO_AIS/climate-forcing/CESM2/LE2/processed/',
              out_folder='/gws/nopw/j04/anthrofail/birgal/NEMO_AIS/boundary-conditions/CESM2/LE2/',
              nemo_mask='/gws/nopw/j04/anthrofail/birgal/NEMO_AIS/bathymetry/mesh_mask-20240305.nc'):
    if dim=='3D':
        name_remapping = {variable:'data', 'TLONG':'lon', 'TLAT':'lat', 'nlat':'y', 'nlon':'x', 'z_t':'z'}
    elif dim=='2D':
        if variable in ['aice','sisnthick','sithick']:
            name_remapping = {variable:'data', 'TLON':'lon', 'TLAT':'lat', 'nj':'y', 'ni':'x'}
        else:
            name_remapping = {variable:'data', 'TLONG':'lon', 'TLAT':'lat', 'nlat':'y', 'nlon':'x'}

    # load NEMO grid:
    nemo_mask_ds = xr.open_dataset(nemo_mask)
    # load CESM2 file and specified month, rename variables:
    cesm2_ds     = xr.open_dataset(f'{cesm2_folder}CESM2-LE2_ens{ens}_{variable}_y{year}.nc')

    if year==1850:
        month_range = range(2,12+1)
    else:
        month_range = range(1,12+1)
    
    for month in month_range:
        cesm2_ds_month = cesm2_ds.isel(time=(cesm2_ds.time.dt.month==month)).isel(time=0)
        if dim=='3D':
            cesm2_ds_month = cesm2_ds_month.assign({'depth':cesm2_ds.z_t})
        cesm2_source = cesm2_ds_month.rename(name_remapping)
    
        # interpolate CESM2 grid to NEMO grid and fill empty values with connected nearest neighbour
        cesm2_interp   = regrid_cesm2_nemo_cf(cesm2_source, variable, dim=dim)
        # extract the boundary layer:
        cesm2_interp   = cesm2_interp.isel(y=slice(-1,None))
        cesm2_extended = fill_ocean(cesm2_interp, variable, nemo_mask_ds.isel(y=slice(-2,-1)), dim=dim, niter=400, fill_val=np.nan, grid=gtype)
        cesm2_extended[variable] = cesm2_extended[variable].roll(x=-1) # because of Fortran based indexing
    
        # Fill land values with an unrealistic constant value and add a time dimension to the shape
        CESM2_bcs = fill_and_reshape(cesm2_extended, variable, land_fill_val=land_fill_val, dim=dim)
    
        # Write output to file:
        CESM2_bcs.to_netcdf(f'{out_folder}CESM2-{exp}_ens{ens}_BCs_{variable}_y{year}m{month:02}.nc', unlimited_dims='time_counter')

    return

In [33]:
for year in range(1857, 1860):
    print('------ year:', year, '------')
    cesm2_bcs('AbsSal', year=year, exp='LE2', ens='1011.001', dim='3D')

Filling gaps with connected nearest neighbours


  4%|▍         | 15/400 [00:00<00:06, 63.27it/s]


Completed filling missing values
Filling gaps with connected nearest neighbours


  4%|▍         | 15/400 [00:00<00:04, 77.31it/s]


Completed filling missing values
Filling gaps with connected nearest neighbours


...

  4%|▍         | 15/400 [00:00<00:04, 86.93it/s]


Completed filling missing values
Filling gaps with connected nearest neighbours


  4%|▍         | 15/400 [00:00<00:04, 84.20it/s]


Completed filling missing values


KeyboardInterrupt: 